<a href="https://colab.research.google.com/github/jyjoon001/gpt2-japanese/blob/master/visual_novel_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visual Novel Generator

이 GPT-2 모델은 사전 학습된 일어 GPT-2 모델에 일본어 텍스트 어드벤처 게임에서 추출된 약 20만줄의 정제된 텍스트 데이터를 전이학습하여 Fine-Tuning하였습니다. 

Pre-Trained Model과 Training을 위한 기반 코드는 [Japanese GPT2 Generation Model](https://github.com/tanreinama/gpt2-japanese) 을 사용하였습니다. 
> [@jyjoon001](https://github.com/jyjoon001) , forked from [tanreinama.](https://github.com/tanreinama) 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/

Mounted at /content/drive
/content/drive/MyDrive


Google Drive에 파일이 있는지 확인하며 파일이 존재하지 않을 경우 Fine-Tune된 모델과 코드를 다운로드합니다. <br> 해당 파일은 1.1GB의 크기를 가지고 있습니다. 
* 적절한 웹호스팅 공간이 없어 임시로 Naver Mail 서버를 사용하고 있습니다.

In [ ]:
import os
if os.path.isfile('/content/drive/MyDrive/visual-novel-generator/gpt2-generate.py') is True:
    print('path found.')
else:
    print('path not found, downloading model.')
    !wget https://bigfile.mail.naver.com/bigfileupload/download?fid=pXR5Wre8D6cZHqujKxgmFoMdax2mHqUmKoumFqg9KxbwFqEdHqurKo2maAvwaxvjF634KzFoMqg9FqUwp6EmF6trF4EZpxFoK6EdFAK9Mov=
    !unzip download?fid=pXR5Wre8D6cZHqujKxgmFoMdax2mHqUmKoumFqg9KxbwFqEdHqurKo2maAvwaxvjF634KzFoMqg9FqUwp6EmF6trF4EZpxFoK6EdFAK9Mov=

path not found, downloading model.
--2021-07-05 09:00:13--  https://bigfile.mail.naver.com/bigfileupload/download?fid=pXR5Wre8D6cZHqujKxgmFoMdax2mHqUmKoumFqg9KxbwFqEdHqurKo2maAvwaxvjF634KzFoMqg9FqUwp6EmF6trF4EZpxFoK6EdFAK9Mov=
Resolving bigfile.mail.naver.com (bigfile.mail.naver.com)... 125.209.234.99
Connecting to bigfile.mail.naver.com (bigfile.mail.naver.com)|125.209.234.99|:443... connected.
HTTP request sent, awaiting response... 200 
Length: 1209179497 (1.1G) [application/x-zip-compressed]
Saving to: ‘download?fid=pXR5Wre8D6cZHqujKxgmFoMdax2mHqUmKoumFqg9KxbwFqEdHqurKo2maAvwaxvjF634KzFoMqg9FqUwp6EmF6trF4EZpxFoK6EdFAK9Mov=’

download?fid=pXR5Wr 100%[===================>]   1.13G  15.2MB/s    in 87s     

2021-07-05 09:01:41 (13.3 MB/s) - ‘download?fid=pXR5Wre8D6cZHqujKxgmFoMdax2mHqUmKoumFqg9KxbwFqEdHqurKo2maAvwaxvjF634KzFoMqg9FqUwp6EmF6trF4EZpxFoK6EdFAK9Mov=’ saved [1209179497/1209179497]

Archive:  download?fid=pXR5Wre8D6cZHqujKxgmFoMdax2mHqUmKoumFqg9KxbwFqEdHqurKo2maAvwaxvjF634Kz

starting_word를 조정해서 생성 모델이 어떤 후속 문장을 생성할지를 정할 수 있습니다.

In [ ]:
starting_word = "けれど、あなたは私に夢と希望をくれました。" #그렇지만, 당신은 저에게 꿈과 희망을 주었습니다.

필요한 코드를 실행시키기 위하여 필요한 라이브러리를 다운받습니다. 
</br> jaconv는 정제되지 않은 일어 데이터를 unicode 데이터로 normalize합니다.

In [ ]:
%cd /content/drive/MyDrive/visual-novel-generator/
with open("prologue.txt", "w") as output:
     output.write(starting_word)
!pip install jaconv

/content/drive/MyDrive/visual-novel-generator
  Created wheel for jaconv: filename=jaconv-0.3-cp37-none-any.whl size=15566 sha256=bf774d249071de64f1e8ef0319cc653d07bde5ad1e046adae46d25c26dd55289
  Stored in directory: /root/.cache/pip/wheels/95/c8/4d/c29135241814c9221027da775d00bcfead63bc924f8ccdaf41
Successfully built jaconv


gpt2-generate.py를 통하여 소스 코드를 실행시킵니다. 
해당 gpt2-generate.py는 다음과 같은 형태로 이루어져 있습니다. 
* argparse를 통하여 argument를 설정합니다. 사용하는 model, 입력/출력 문장, 최대/최소 음절과 문장이 얼마나 임의의 단어를 생성하는지에 대한 temperature 계수, top-k 및 top-p sampling 시 사용하는 어절의 갯수 등을 결정합니다. 
* 사용하는 Model의 정보를 불러온 뒤 layer 갯수 등의 정보를 결정합니다. 
* generate_one 함수에서는 우선 입력 문장을 encode하여  Tensorflow session을 실행시켜 후속 토큰을 생성하며, 이를 decode하여 자연어로 출력한다. 이 때```'<|endoftext|>'``` 토큰이 나올 경우는 생성된 문장에 last 라벨링을 하여 문장의 끝을 알 수 있게 합니다. 
* Tensorflow session의 경우 sampling.py의 sample_sequence를 통해서 output을 결정합니다. 이후, max_length까지 위의 함수를 통하여 문장을 generate합니다. 

In [ ]:
!python3 gpt2-generate.py --num_generate 1 --temperature 1.5 --max_length 1024 --min_length 512
from google.colab import files
files.download("script.txt")

2021-07-05 09:03:03.645205: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-07-05 09:03:05.026869: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-07-05 09:03:05.085617: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-07-05 09:03:05.086211: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-07-05 09:03:05.086262: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-07-05 09:03:05.268662: I tensorflow/stream_executor/platform/default

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>